In [ ]:
import numpy as np
import cv2

In [ ]:
def common_data(list1,list2):
    if list2 == []:
        return -1,-1 # add all
    if list1 in list2:
        return -2,-2
    m=list1[0]
    n=list1[1]
    i,j=0,0
    for i in range(0,len(list2)):
        if(m in list2[i]):
            if(n in list2[i]):
                return -2,-2 # no action
            else:
                return n,i
        if(n in list2[i]):
            if(m in list2[i]):
                return -2,-2
            else:
                return m,i
    return -1,-1

In [ ]:
def find_connected_components(img):
    connected=[]
    temp=[]
    pixel=0
    m,n = img.shape
    for i in range(0,m):
        new = False
        for j in range(0,n):
            if(img[i][j] == 255):
                if(new == False):
                    if(img[i-1][j] !=0):
                        img[i][j] = img[i-1][j]
                    else:
                        pixel += 1
                        img[i][j] = pixel
                    new = True
                else:
                    img[i][j] = max(img[i][j-1],img[i-1][j])
                    if(img[i][j-1]!=0 and img[i-1][j] !=0 and img[i][j-1] != img[i-1][j]):
                        temp.append(img[i][j-1])
                        temp.append(img[i-1][j])
                        value,index = common_data(temp,connected)
                        if value == -1:
                            connected.append(temp)
                        elif index > 0:
                            connected[index].append(value)
                        temp=[]
            else:
                new = False
    return len(connected)

In [ ]:
def convert_image(img):
    m,n,p = img.shape
    newImg = np.zeros((m,n))
    for i in range(0,m-1):
        for j in range(0,n-1):
            if(240>img[i][j][2]>200 and 230>img[i][j][1]>200 and 250>img[i][j][0]>220):
                newImg[i][j] = 255
    return newImg

In [ ]:
def errosion(img,x,y,blockSize):
    for j in range(0,blockSize):
        for i in range(0,blockSize):
            if(img[x+j][y+i] != 255):
                return 0
    return 255

In [ ]:
def dilation(img,x,y,blockSize):
    for j in range(0,blockSize):
        for i in range(0,blockSize):
            if((x+j) < len(img) and (y+i) < len(img[0])):
                if(img[x+j][y+i] == 255):
                    return 255
    return 0

In [ ]:
def apply_morphologicalOperation(img, blockSize,operation):
    m,n = img.shape
    newImg = np.zeros((m,n))
    for i in range(0,m-1):
        for j in range(0,n-1):
            if(operation == "errosion"):
                newImg[i][j] = errosion(img,i,j,blockSize)
            if(operation == "dilation"):
                newImg[i][j] = dilation(img,i,j,blockSize)
    return newImg

In [ ]:
def Find_threshold(img,threshold):
    group1=[]
    group2=[]
    for i in img:
        for j in i:
            if j > threshold:
                group2.append(j)
            else:
                group1.append(j)
    avg1 = sum(group1)/len(group1)
    avg2 = sum(group2)/len(group2)
    new_threshold = (avg1+avg2)/2
    #print(new_threshold)
    if(new_threshold == threshold):
        return new_threshold
    return Find_threshold(img,new_threshold)

In [ ]:
def findEdges(img):
    newImg = np.zeros((len(img),len(img[0])))
    matrix_y = [[1,2,1],
        [0,0,0],
        [-1,-2,-1]]
    matrix_x = [[1,0,-1],
        [2,0,-2],
        [1,0,-1]]
    threshold = Find_threshold(img,0)
    for x in range(0,(len(img)-1)):
        for y in range(0,(len(img[0])-1)):
            s= [[img[x-1][y-1],img[x][y-1],img[x+1][y-1]],
                [img[x-1][y],img[x][y],img[x+1][y]],
                [img[x-1][y+1],img[x][y+1],img[x+1][y+1]]]
            gradient_x = calculateGradient(s,matrix_x)
            gradient_y = calculateGradient(s,matrix_y)
            gradient = ((gradient_x **2)+(gradient_y**2))**0.5
            if(gradient > threshold):
                newImg[x][y] = 255
    return newImg

In [ ]:
def calculateGradient(matrix1,matrix2):
    s=0
    for x,y in zip(matrix1,matrix2):
        for a,b in zip(x,y):
            s = s+(a*b)
    return s

In [ ]:
def labelImg(img,edgedImg):
    m,n = edgedImg.shape
    for i in range(0,m-1):
        for j in range(0,n-1):
            if(edgedImg[i][j] == 255):
                img[i][j] = 0
    return img

In [ ]:
img = cv2.imread("Picture3-small.bmp")
abnormal_cells = convert_image(np.copy(img))
errotedImg = apply_morphologicalOperation(abnormal_cells,4,"errosion")
dilatedImg = apply_morphologicalOperation(errotedImg,5,"dilation")
edgedImg = findEdges(np.copy(dilatedImg))
labeledImg = labelImg(img,edgedImg)
cv2.imwrite("labeledImg.jpg",labeledImg)

True

In [ ]:
components = find_connected_components(dilatedImg)

In [ ]:
print("Number of abnormal cells is "+str(components))

Number of abnormal cells is 37
